In [3]:
# Only do this install once, for experimenting with hugging face's transformers
# CPU usage, just to ensure that model architecture will be working.
# Will later likely need to use GPUs using a virtual environment,
# IF: our dataset is too big, possibly not the case.
#pip install transformers[torch]

# Same here, run once
#!pip install datasets

   ---------------------------------------- 0.0/542.0 kB ? eta -:--:--
   ---------------------- ---------------- 317.4/542.0 kB 19.2 MB/s eta 0:00:01
   ---------------------------------------- 542.0/542.0 kB 8.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/371.6 kB ? eta -:--:--
   -------------------------------- ------ 307.2/371.6 kB 18.6 MB/s eta 0:00:01
   ---------------------------------------- 371.6/371.6 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
   - -------------------------------------- 0.7/25.9 MB 22.4 MB/s eta 0:00:02
   - -------------------------------------- 1.2/25.9 MB 14.7 MB/s eta 0:00:02
   - -------------------------------------- 1.2/25.9 MB 14.7 MB/s eta 0:00:02
   - -------------------------------------- 1.2/25.9 MB 14.7 MB/s eta 0:00:02
   - 

In [1]:
# imports need necessary installs found above

from naiveModel import NBLangIDModel
from BERTModel import BERTGenreClassification, train_model
import pandas as pd
from sklearn.model_selection import train_test_split
from util import get_dataloader

# don't need cuda until using virtual machine
from torch import manual_seed#, cuda

c:\Users\oscar\anaconda3\envs\ml-0451\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
naiveBayes = NBLangIDModel()

In [15]:
BERT = BERTGenreClassification()

In [16]:
# initialize model and dataloaders

# change later, if using virtual machine
device ='cpu' #"cuda" if cuda.is_available() else "cpu"

# seed the model before initializing weights so that your code is deterministic
manual_seed(457)

freeze_bert = False # change later
batch_size = 16 # default
epochs = 5 # default
learning_rate = 1e-2 # default

model = BERTGenreClassification(freeze_bert = freeze_bert).to(device)

In [17]:
# load data, train test split
descriptions = pd.read_csv("cleanedData.csv")

print("Shape before dropping NaN values:", descriptions.shape)
descriptions = descriptions.dropna()
print("Shape after dropping NaN values:", descriptions.shape)

#TESTING THIS
# print("Rows to be dropped:")
# print(descriptions[descriptions['description'].isna() | ~descriptions['description'].apply(lambda x: isinstance(x, str))])
# #drop the 511 rows where description column is NaN or not a string
# descriptions = descriptions.dropna(subset=['description'])
# descriptions = descriptions[descriptions['description'].apply(lambda x: isinstance(x, str))]

train, test = train_test_split(descriptions, test_size=0.2)

Shape before dropping NaN values: (42661, 5)
Shape after dropping NaN values: (37478, 5)


In [18]:
train.drop("Unnamed: 0", axis= 1)
test.drop("Unnamed: 0", axis= 1)

,description,genre1,genre2,genre3
39832,Growing up in a small upper Midwestern town in...,'Fiction','Literature','Coming Of Age'
11687,Alternate cover edition of ASIN B005TVVS2EMord...,'Fantasy','Magic','Epic Fantasy'
19420,Creativity is fundamental to human experience....,'Philosophy','Nonfiction','Science'
21964,"Lucas had it all—popularity, a devoted girlfri...",'Young Adult','Romance','New Adult'
35393,"Frida Kahlo, painter and cultural icon, lived ...",'Fiction','Art','Historical Fiction'
...,...,...,...,...
24216,The first volume of The Heart Rate of a Mouse ...,'Romance','LGBT','Fan Fiction'
41649,When Miss Charlotte Wilson asks God for a husb...,'Romance','Christian Fiction','Historical Fiction'
4386,In Catherine Anderson’s newest installment in ...,'Romance','Contemporary','Contemporary Romance'
13378,Illustrates outdoor sculptures created with a ...,'Art','Nonfiction','Photography'


In [19]:
train_X = train['description']
train_y1 = train['genre1']
train_y2 = train['genre2']
train_y3 = train['genre3']

In [26]:
# fit the NB model 
naiveBayes.fit(train_X.tolist(), train_y1.tolist())

In [28]:
label_vocab = naiveBayes.labels
print(label_vocab)

label_as_id = {l:k  for k, l in enumerate(label_vocab)}

{"'Anthologies'", "'Cookbooks'", "'Dogs'", "'Interracial Romance'", "'Game Design'", "'Martial Arts'", "'Westerns'", "'Philosophy'", "'Art'", "'Teaching'", "'Photography'", "'Romantic'", "'Medieval'", "'Novels'", "'Evolution'", "'Writing'", "'Novella'", "'Essays'", "'Nonfiction'", "'Marriage'", "'Star Wars'", "'Zombies'", "'Environment'", "'Birds'", "'Indian Literature'", "'Science Fiction'", "'Theology'", "'Finance'", "'Computers'", "'Poland'", "'Adoption'", "'Autobiography'", "'Dinosaurs'", "'Buffy The Vampire Slayer'", "'Memoir'", "'Cats'", "'True Story'", "'Reverse Harem'", "'Book Club'", "'Asexual'", "'Self Help'", "'Fairies'", "'Survival'", "'Abuse'", "'Science Fiction Romance'", "'Werewolves'", "'Russia'", "'Theatre'", "'Music'", "'Travel'", "'Education'", "'Egypt'", "'German Literature'", "'Romantic Suspense'", "'Polish Literature'", "'Architecture'", "'Judaism'", "'Lds Fiction'", "'Queer'", "'Sociology'", "'Lds'", "'Suspense'", "'Indonesian Literature'", "'Amish'", "'India'", 

In [13]:
bert_train, bert_val = train_test_split(train, test_size=0.2)

In [ ]:
# BERT data loaders and training

# have to change these to implement from df, not from csv
train_dataloader = get_dataloader(bert_train, batch_size=batch_size)
val_dataloader = get_dataloader(bert_val, batch_size=batch_size)


# currently, embedding dimensions are wrong
#train_model(model, train_dataloader, dev_dataloader, epochs, learning_rate)